# Canon DLSR Scraper

### Importing Libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from more_itertools import unique_everseen
from pprint import pprint
from wordcloud import WordCloud
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from collections import OrderedDict
from datetime import date
import time
import csv
import datetime as dt
import warnings
from fuzzywuzzy import fuzz
warnings.filterwarnings('ignore')

### Starting the WebDriver

In [2]:
#Starting the driver and finding the query element
driver = webdriver.Chrome()
driver.get("https://washingtondc.craigslist.org/nva")
search = driver.find_element_by_id("query")

In [3]:
#Inputing the search keyword
search.send_keys("Canon")
search.send_keys(Keys.ENTER)

In [4]:
#Selecting the photo/video and electronics cateories
driver.find_element_by_css_selector('input.catcheck.selectallcb').click()
driver.find_element_by_css_selector(".catcheck.multi_checkbox[id='cat_pha']").click()
driver.find_element_by_css_selector(".catcheck.multi_checkbox[id='cat_ela']").click()
driver.find_element_by_css_selector("button.searchlink.linklike.changed_input.clickme").click()

#Bundling duplicates, requiring pictures, not including nearby searches
driver.find_element_by_name("bundleDuplicates").click()
driver.find_element_by_name("hasPic").click()

In [5]:
#Inputing the min and max price
minP = driver.find_element_by_name("min_price")
minP.send_keys(500)
time.sleep(1)
maxP = driver.find_element_by_name("max_price")
maxP.send_keys(1500)
maxP.send_keys(Keys.ENTER)

### Gathering Eligible Links

In [6]:
#Prepping for beautiful soup
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [7]:
#Going through all content divs and finding the links
nduplinks = []
duplinks=[] 

#Finding the non duplicated results
nonDups = soup.findAll('li', {'class': 'result-row'})
for item in nonDups:
    nduplinks.append(item.find('a').get('href'))
    
#Finding the duplicated results
dups = soup.findAll('ul', {'class': 'duplicate-rows'})
for item in dups:
    duplinks.append(item.find('a').get('href'))

In [8]:
#Removing the duplicates, returning the links list
links = [x for x in nduplinks if x not in duplinks]

#Removing duplicate links due to pictures and titles both being the same
links = list(unique_everseen(links))
while "#" in links:
    links.remove("#")

In [9]:
#Clicking into the item links
count = 0
dailyitems=[]
itemDict = []
for link in links:
    #Clicking the item link
    link = '"'+link+'"'
    element = driver.find_element_by_xpath('//a[@href=%s]' %link)
    driver.execute_script('arguments[0].click();', element)

    #Getting the item html for beautiful soup
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    #Getting the item price and title
    price = soup.find('span', {'class': 'price'}).get_text()
    title = soup.find('span', {'id': 'titletextonly'}).get_text()
    
    #Getting the item text
    text = soup.find('section', {'id':'postingbody'}).get_text()
    while "\n" in text:
        text = text.replace("\n", "")
    while "QR Code Link to This Post" in text:
        text = text.replace("QR Code Link to This Post", "")
    itemD = {"Title":title, "Price":price, "Description":text}

    #Making a list of dictionaries
    itemDict.append(itemD)

    #Adding to a list
    dailyitems.append([itemD['Title'], itemD['Price'], itemD['Description']])
    count += 1
    driver.back()

In [10]:
#Outputting the results array item by item
driver.close()

#Creating a dataframe of the items
current_listings = pd.DataFrame(itemDict)

In [11]:
#Removing the dollar sign in current listings:
if current_listings.Price.dtypes.name != 'int64':
    current_listings['Price'] = pd.to_numeric(current_listings['Price'].str.replace('$', ''))

#Outputting current listings
current_listings = current_listings[['Title', 'Description', 'Price']]

In [12]:
#Recording the Time the Script Finished Crawling
time_ran = dt.datetime.now()

# Update Current Listings and Sold Items

### Read in Current Listings from Yesterday

In [13]:
#Loading the listings from the day before (Dataframe1)
old_listings = pd.read_csv('C:/Users/mikes/Documents/Python Scripts/Canon_Scraper/Data_Tables/CanonCurrentListings.csv')

### Merge Yesterday's and Today's Listings With the Same Title and Description

In [14]:
common = old_listings.merge(current_listings, on=['Title', 'Description'])
common['Current_Price'] = common['Price']
common.drop('Price', axis=1, inplace=True)

### Consider Cases Where the Description Has Been Changed Slightly

In [15]:
#Finding today's listings not perfectly matched in yesterdays listings
compDF = current_listings.merge(old_listings, indicator=True, how='outer', on=['Title', 'Description'])
only_today = compDF[compDF['_merge'] == 'left_only']

#Finding items in yesterdays listings not perfectly matched in todays listings
only_old = compDF[compDF['_merge'] == 'right_only']

For each item that is only in yesterdays listings, we need to compare the similarity of description to every item that is only in today's listing. If the fuzz ratio is greater than 85 (out of 100), we will consider the items a match and merge the old and new values.  

In [16]:
#Storing the matched listings between today and old only
cols = ['Current_Price','Description','List_Date','List_Price','Title']
matches = pd.DataFrame(columns=cols)

#Performing the matching
for old_index, old_row in only_old.iterrows():
    for new_index, new_row in only_today.iterrows():
        if fuzz.ratio(old_row.Description, new_row.Description) > 85:
            #Add listing to matches dataframe
            matches = matches.append({'Current_Price':new_row.Price, 
                            'Description':new_row.Description, 
                            'List_Date':old_row.List_Date,
                            'List_Price':old_row.List_Price, 
                            'Title':new_row.Title}, ignore_index=True)
            
            #Remove item from old_only
            only_old = only_old[~(only_old.Description == old_row.Description)]
            
            #Remove item from new_only
            only_today = only_today[~(only_today.Description == new_row.Description)]
         
            #Break out of the inner looping
            break

In [17]:
#Add the matches to the common listings
common = common.append(matches)

### Adding New Items to Current Listings

In [18]:
#Adding the list day to new items dataframe
new_items = only_today[['Title', 'Description', 'Price']]

new_items['List_Date'] = time_ran.date()

#Renaming price to be List Price 
new_items.rename(columns = {'Price':'List_Price'}, inplace=True)

#Reflect the Current Price to be equal to the list price
new_items['Current_Price'] = new_items['List_Price']

#Adding new items to the common listings
today_listings = common.append(new_items)

In [19]:
#Writing the listings to csv to be used tomorrow
today_listings.to_csv('C:/Users/mikes/Documents/Python Scripts/Canon_Scraper/Data_Tables/CanonCurrentListings.csv', index=False, encoding='utf-8')

### Addding Items Sold Today to Sold Items

In [20]:
sold_today = only_old

#Updating the historical table of sold items
if sold_today.shape[0] != 0:
    sold_today.drop(['_merge', 'Price'], axis=1, inplace=True)
    sold_today.rename(columns={'Current_Price':'Sell_Price'}, inplace=True)
    sold_today['Sell_Date'] = time_ran.date()
    
    #Loading the sold items csv
    sold_items = pd.read_csv('C:/Users/mikes/Documents/Python Scripts/Canon_Scraper/Data_Tables/SoldItems.csv')
    
    #Adding items sold today to sold_items
    sold_items = sold_items.append(sold_today)
    
    #Writing to the sold items csv
    sold_items.to_csv('C:/Users/mikes/Documents/Python Scripts/Canon_Scraper/Data_Tables/SoldItems.csv', index=False, encoding='utf-8')